# MTH 4224 / CSE 4224 - Python Assignment

## Problem 1: Numerical Integration

Design a Python class `NumericalIntegrator` that provides methods for numerically integrating a function of one variable over a specified interval. Requirements:

1. [5 points] Include an `__init__` function to initialize `NumericalIntegrator` objects with user-specified function to integrate, and the interval to integrate.

2. [10 points]  Include methods to use the (a) left-hand rule, (b) right-hand rule, (c) midpoint rule, (d) trapezoidal rule, (e) Simpson's rule, and (f) a Monte Carlo approach, with all necessary parameters as inputs.

3. [5 points]  Include methods to measure the error compared to an explicit solution and runtime.

4. [5 points]  For the following functions, perform an integral by hand to find an explicit solution and then compare the error rates and runtimes of each of the 5 numerical methods above. (Choose the parameters to enable a fair comparison.)

   * $3x^3-2x+4$ on the interval $[-1,1]$
   * $\sin^n x$ on the interval $[0,\pi]$ (test for $n=1,2,3,5,10$) [*Hint.* Review reduction formulas from Calculus 2.]
   * $e^x\sin x$ on the interval $[0,\pi]$
   * $\frac{1}{x^2}$ on the interval $[1,\infty)$

In [5]:
import numpy as np
import time

class NumericalIntegrator:
    # initialize function and endpoints
    def __init__(self, func, a, b):
        self.func = func
        self.a = a
        self.b = b
    
    # integration method (default to midpoint rule)
    def integrate(self, method='midpoint', n=100):
        if method == 'left':
            return self.left_rule(n)
        elif method == 'right':
            return self.right_rule(n)
        elif method == 'midpoint':
            return self.midpoint_rule(n)
        elif method == 'trapezoidal':
            return self.trapezoidal_rule(n)
        elif method == 'simpson':
            return self.simpson_rule(n)
        elif method == 'monte carlo':
            return self.monte_carlo(n)
        else:
            raise ValueError("Method not recognized.")
    
    # left hand rule
    def left_rule(self, n):
        dx = (self.b - self.a) / n
        x = np.linspace(self.a, self.b - dx, n)
        return np.sum(self.func(x) * dx)
    
    # right hand rule
    def right_rule(self, n):
        dx = (self.b - self.a) / n
        x = np.linspace(self.a + dx, self.b, n)
        return np.sum(self.func(x) * dx)
    
    # midpoint rule
    def midpoint_rule(self, n):
        dx = (self.b - self.a) / n
        x = np.linspace(self.a + dx/2, self.b - dx/2, n)
        return np.sum(self.func(x) * dx)
    
    # trapezoidal rule
    def trapezoidal_rule(self, n):
        dx = (self.b - self.a) / n
        x = np.linspace(self.a, self.b, n+1)
        y = self.func(x)
        return dx * (np.sum(y) - 0.5 * (y[0] + y[-1]))
    
    # Simpson's rule
    def simpson_rule(self, n):
        if n % 2 == 1: n += 1  # Simpson's rule needs an even number of intervals
        dx = (self.b - self.a) / n
        x = np.linspace(self.a, self.b, n+1)
        y = self.func(x)
        return dx/3 * np.sum(y[0:-1:2] + 4*y[1::2] + y[2::2])
    
    # Monte Carlo method
    def monte_carlo(self, n):
        x = np.random.uniform(self.a, self.b, n)
        return (self.b - self.a) * np.mean(self.func(x))
    
    # assessment function
    def evaluate_error_and_runtime(self, method, n, exact_solution):
        start_time = time.perf_counter()
        result = self.integrate(method, n)
        end_time = time.perf_counter()
        error = abs(result - exact_solution)
        runtime = (end_time - start_time) * 1e6  # Convert seconds to microseconds
        return result, error, runtime

### Explicit Integration

#### Integral 1

$$\int\limits_{-1}^1 3x^3-2x+4\,dx=\left[\frac{3}{4}x^4-x^2+4x\right]_{-1}^1 = \left(\frac{3}{4}-1+4\right) - \left(\frac{3}{4}-1-4\right) = 8$$

#### Integral 2

$$\int_0^\pi \sin^n x\,dx$$

For $n=1$,

$$\int_0^\pi \sin x\,dx = \left[-\cos x\right]_0^\pi= -(-1) - (-1)=2$$

For $n>1$, let's use integration by parts $\int u\,dv=uv - \int v\,du$, where $u=\sin^{n-1}x$ and $dv=\sin x\,dx$, so we have $du=(n-1)\sin^{n-2}x\cos x\,dx$ and $v=-\cos x$, then

$$\begin{align*}
\int_0^\pi \sin^n x\,dx&=-\left.\sin^{n-1}x\cos x\right]_0^\pi-\int_0^\pi (n-1)\sin^{n-2} x\left(-\cos^2 x\right)\,dx
\\ \int_0^\pi \sin^n x\,dx&=(n-1)\int_0^\pi \sin^{n-2} x\left(1 - \sin^2 x\right)\,dx
\\ \int_0^\pi \sin^n x\,dx&=(n-1)\int_0^\pi \sin^{n-2} x\,dx-(n-1)\int_0^\pi \sin^n x\,dx
\\ \int_0^\pi \sin^n x\,dx&=(n-1)\int_0^\pi \sin^{n-2} x\,dx-(n-1)\int_0^\pi \sin^n x\,dx
\\ n\int_0^\pi \sin^n x\,dx&=(n-1)\int_0^\pi \sin^{n-2} x\,dx
\\ \int_0^\pi \sin^n x\,dx&=\frac{n-1}{n}\int_0^\pi \sin^{n-2} x\,dx
\end{align*}$$

This formula can be used iteratively. Noting $\int_0^\pi\sin^1 x\,dx=2$ and $\int_0^\pi \sin^0 x\,dx=\int_0^\pi dx=\pi$, solving them is now simple.

For $n=2$,

$$\int_0^\pi\sin^2 x\,dx=\frac{1}{2}\int_0^\pi\sin^0 x\,dx=\frac{\pi}{2}$$

For $n=3$,

$$\int_0^\pi\sin^3 x\,dx=\frac{2}{3}\int_0^\pi\sin^1 x\,dx=\frac{4}{3}$$

For $n=5$,

$$\int_0^\pi\sin^5 x\,dx=\frac{4}{5}\int_0^\pi\sin^3 x\,dx=\frac{4}{5}\frac{2}{3}\int_0^\pi \sin x\,dx = \frac{16}{15}$$

For $n=10$,

$$\begin{align*}
\int_0^\pi\sin^{10} x\,dx &=\frac{9}{10}\int_0^\pi\sin^8 x\,dx
\\&=\frac{9}{10}\frac{7}{8}\int_0^\pi\sin^6 x\,dx
\\&=\frac{9}{10}\frac{7}{8}\frac{5}{6}\int_0^\pi\sin^4 x\,dx
\\&=\frac{9}{10}\frac{7}{8}\frac{5}{6}\frac{3}{4}\int_0^\pi\sin^2 x\,dx
\\&=\frac{9}{10}\frac{7}{8}\frac{5}{6}\frac{3}{4}\frac{1}{2}\int_0^\pi dx
\\&=\frac{63}{256}\pi
\end{align*}$$

#### Integral 3

Let $u=e^x$ and $dv=\sin x\,dx$, then $du=e^x\,dx$ and $v=-\cos x$. Using integration by parts $\int u\,dv=uv - \int v\,du$, we have

$$\begin{align*}
\int_0^\pi e^x\sin x\,dx&=\left.-e^x\cos x\right]_0^\pi - \int_0^\pi -\cos x e^x\,dx
\\&= -e^{\pi}\cos \pi + e^0\cos 0 + \int_0^\pi e^x\cos x
\\&= 1+e^\pi + \int_0^\pi e^x\cos x
\end{align*}$$

Next, let's apply integration by parts to the remaining integral with $u=e^x$, $du=e^x\,dx$, $dv=\cos x\,dx$, and $v=\sin x$:

$$\begin{align*}
\int_0^\pi e^x\sin x\,dx&= 1+e^\pi + \left[e^x\sin x\right]_0^\pi - \int_0^\pi e^x\sin x\,dx
\\2\int_0^\pi e^x\sin x\,dx&=1+e^\pi + 0 - 0
\\ \int_0^\pi e^x\sin x\,dx&=\frac{1+e^\pi}{2}
\end{align*}$$

#### Integral 4

$$\int_1^\infty \frac{1}{x^2}\,dx=\left[-\frac{1}{x}\right]_1^\infty=\lim\limits_{x\to\infty} -\frac{1}{x} + \frac{1}{1}=0+1=1$$

### Numerical Integration Performance Measures

In [ ]:

poly = lambda x: 3*x**3 - 2*x + 4
sin1 = lambda x: np.sin(x)
sin2 = lambda x: np.sin(x)**2
sin3 = lambda x: np.sin(x)**3
sin5 = lambda x: np.sin(x)**5
sin10 = lambda x: np.sin(x)**10
esin = lambda x: np.exp(x)*np.sin(x)
xn2 = lambda x: 1/x**2

functions = [poly, sin1, sin2, sin3, sin5, sin10, esin, xn2]
names = ['3x^3 - 2x + 4', 'sin x', 'sin^2 x', 'sin^3 x', 'sin^5 x', 'sin^10 x', 'e^x sin x', '1/x^2']
a_s = [-1., 0., 0., 0., 0., 0., 0., 1.]
b_s = [1., np.pi, np.pi, np.pi, np.pi, np.pi, np.pi, 10.]

exact_integrals = [8., 2., np.pi/2, 4/3, 16/15, 63*np.pi/256, (1+np.exp(np.pi))/2, 1.]

for function, name, exact, a, b in zip(functions, names, exact_integrals, a_s, b_s):
    print('\n================================================================================================================')
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<20}'.format('Method', 'Approx.', 'Exact', 'Error', 'Rel. Error', 'Runtime (µs)'))
    print('================================================================================================================')

    print(f'\nIntegral of {name} from {a} to {b}: Exact = {exact}\n')
    print('----------------------------------------------------------------------------------------------------------------')
    
    for method in ['left', 'right', 'midpoint', 'trapezoidal', 'simpson', 'monte carlo']:
        numerical_integrator = NumericalIntegrator(function, a, b)
        result, error, runtime = numerical_integrator.evaluate_error_and_runtime(method, 1000, exact)
        rel_error = error/result
        
        print('{:<15} {:<15.4e} {:<15.4e} {:<15.4e} {:<15.4e} {:<20.0f}'.format(method, result, exact, error, rel_error, runtime))

print('\n================================================================================================================')



Method          Approx.         Exact           Error           Rel. Error      Runtime (µs)        

Integral of 3x^3 - 2x + 4 from -1.0 to 1.0: Exact = 8.0

----------------------------------------------------------------------------------------------------------------
left            7.9980e+00      8.0000e+00      2.0000e-03      2.5006e-04      592                 
right           8.0020e+00      8.0000e+00      2.0000e-03      2.4994e-04      122                 
midpoint        8.0000e+00      8.0000e+00      0.0000e+00      0.0000e+00      98                  
trapezoidal     8.0000e+00      8.0000e+00      0.0000e+00      0.0000e+00      92                  
simpson         8.0000e+00      8.0000e+00      0.0000e+00      0.0000e+00      137                 
monte carlo     7.9585e+00      8.0000e+00      4.1539e-02      5.2195e-03      140                 

Method          Approx.         Exact           Error           Rel. Error      Runtime (µs)        

Integral of sin x 


## Problem 2: Ordinary Least Squares on Real Data

1. [10 points] Read the WHO Life Expectancy dataset (https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who) into Python and clean it into a data matrix $X$ with as many features as possible and a life expectancy label vector $Y$. Find a solution for dealing with missing data, convert all relevant data to numerical data using best practices.

2. [5 points] Split the data randomly into 80\% training data and 20\% test data. Train an OLS model on the training data and test on the test data. Report the sum of squared errors and runtime for both training and inference.

In [37]:
import pandas as pd

data = pd.read_csv('../data/Life Expectancy Data.csv').drop('Country', axis=1)
data = data.dropna()
data = pd.get_dummies(data, columns=['Status'])
data = data.astype('float64')

Y = data['Life expectancy '].to_numpy()
X = data.drop('Life expectancy ', axis=1).to_numpy()

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train OLS model
model = LinearRegression()
start_time = time.time()
model.fit(X_train, y_train)
training_time = time.time() - start_time
y_train_pred = model.predict(X_train)

# Test the model
start_time = time.time()
y_pred = model.predict(X_test)
testing_time = time.time() - start_time

# Compute error
training_error = mean_squared_error(y_train, y_train_pred)
testing_error = mean_squared_error(y_test, y_pred)

print(f'Training time: {training_time}s')
print(f'Inference time: {testing_time}s\n')

print(f'Training Mean squared error: {training_error}')
print(f'Testing Mean squared error: {testing_error}')

Training time: 0.00555109977722168s
Inference time: 0.00024771690368652344s

Training Mean squared error: 12.41538216037127
Testing Mean squared error: 13.014944111864585
